# Prediction Visualizations

- take the best model
- do the following on training data and then testing data
- see the distribution of predicted classes (how many were predicted 1, 2, and 3?)
- see only where the predictions were incorrect
- see any patterns in wrong predictions (ex: when actual answer was 1, it always predicted 2)
- then create custom error rates 
- continue creating custom metrics to give insights into model weaknesses

In [1]:
import joblib

In [ ]:
dt_model = joblib.load('../models/best_decision_tree_model.joblib')
knn_model = joblib.load('../models/best_knn_model.joblib')
best_rf_model = joblib.load('../models/best_random_forest_model.joblib')

In [ ]:
model = best_rf_model
y_pred = model.predict(x_test)
print(y_pred[:5])
print(y_test[:5])